In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
## This function allows to execute the OS commands
def xcmd(cmd, verbose=True):
    """Runs an OS command
    :param cmd: terminal command
    :type cmd: ``str``
    :param verbose: printing the details, default True 
    :type verbose: ``boolean``
    :return: OS outputs
    :rtype: ``str``
    """

    if verbose: print('\n'+cmd)

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if False:
        print('execution of %s failed' % cmd)
        print('error is as follows', output)
        sys.exit()
    else:
        return output


In [3]:
# This function works with both dbtable and the catalog header name on EDD
def get_catal(catal_string, infoFile="../bar_files/catalogs_info.dat", bars_folder='../bar_files/'):
    
    xcmd("scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.", verbose=True)

    with open(infoFile, "r") as f:
        lines = f.readlines()


        try:
            for i, line in enumerate(lines):
                if catal_string in line:
                    
                    while not "begin" in line:
                        i-=1
                        line = lines[i]
                    while not "filename" in line:
                        i+=1
                        line=lines[i]

                    fname_string = line
                    break
        except: 
            return "", []
            
        catal_columns = []
        
        try:
            while not "column" in line:
                i+=1
                line=lines[i]
            while not 'end' in line:
                catal_columns.append(line.split("=>")[1].split("|")[0].strip())
                i+=1
                line=lines[i]
                
        except:
            print("why")
            return "", []


    catal_file_name = fname_string.split("=>")[1].strip("\n").strip("")
    
    xcmd("scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/"+catal_file_name+"  "+bars_folder+"/.", verbose=True)
    
    data = pd.read_csv(bars_folder+catal_file_name, names=catal_columns, delimiter='|')
    
    for col in catal_columns:
        if col.upper()=="PGC":
            break
    
    data.rename(columns={col:col.upper()}, inplace=True)
    data = data.set_index('PGC')
    
    # taking care of empty string
    data = data.replace(r'^\s*$', np.nan, regex=True)
    
    if "1PGC" in data.columns:
        data = data.rename(columns={"1PGC": "PGC1"})
    else:
        data["PGC1"] = data.index.values
    
    if "av_flag" in data.columns:
        data = data[data.av_flag==0]
    
    
    return data

In [4]:
def make_data(data, catalogs, catal_data, catal_key, catal_DM="DM", catal_eDM="eDM", delta=0, \
              cols=["Vcmb","Nest"]):
    
    if not catal_key in catalogs and not catal_key in data:
        data[catal_key] = catal_data[["PGC1", catal_DM, catal_eDM]+cols]
        data[catal_key] = data[catal_key].rename(columns={"PGC1": "PGC1_"+catal_key})
        data[catal_key] = data[catal_key].rename(columns={catal_DM: "DM_"+catal_key})
        data[catal_key] = data[catal_key].rename(columns={catal_eDM: "eDM_"+catal_key})
        
        for col in cols:
            data[catal_key] = data[catal_key].rename(columns={col: col+"_"+catal_key})
        
        catalogs.append(catal_key)
        
        data[catal_key]["DM_"+catal_key] += delta
        
    else:
        print("This catalog has been already imported ...")
        print("Use a different key than "+catal_key)
    
    return data, catalogs
    

In [5]:
#################################################################
def Vh2V3k(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    v3k = Vh-25.2*cosl*cosb-245.7*sinl*cosb+276.8*sinb

    return v3k

#################################################################

def Vcmb2Vmod(Vcmb, omegam=0.27):
    
    omegal=1.-omegam
    c=299800.
    z=Vcmb/c
    q0=0.5*(omegam-2.*omegal)
    fmod=1.+0.5*(1.-q0)*z-(1./6)*(1.-q0-3.*q0**2+1.)*z**2
    Vmod=c*z*fmod

    return Vmod
#################################################################

def Vh2Vls(el,b, Vh):
  
    alpha = np.pi / 180.
    cosb = np.cos(b*alpha)
    sinb = np.sin(b*alpha)
    cosl = np.cos(el*alpha)
    sinl = np.sin(el*alpha)
    
    vls = Vh-26.*cosl*cosb+317.*sinl*cosb-8.*sinb

    return vls
#################################################################

In [6]:
data = {}
catalogs = []

In [7]:
catal_dbtable = "k6dfgsrev"
catal = get_catal(catal_dbtable)

catal  = catal[catal['rej']==0]

data, catalogs = make_data(data, catalogs, catal, "6dfgs", catal_DM="DM75", catal_eDM="eDM")
catal.head()


scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.

scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/6dFGSv_recal_bar  ../bar_files//.


,rej,PGC1,Nest,Name,Vcmb,Vgp,D75,D100,feD,DM75,DM100,eDM,RA,DE,Vg,Gp
PGC,,,,,,,,,,,,,,,,
64,0,72642,200033,g0000523-355037,15324,14646,219.3,164.5,0.29,36.705,36.080,0.551,0.2181,-35.8437,1261,NaN
66,0,72642,200033,g0000532-355911,14725,14646,188.7,141.5,0.27,36.379,35.754,0.519,0.2216,-35.9863,1261,NaN
114,0,72642,200033,g0001341-361900,14397,14397,132.1,99.0,0.27,35.604,34.979,0.526,0.3923,-36.3167,-1,NaN
115,0,115,0,g0001453-042049,14005,14005,186.5,139.8,0.27,36.353,35.728,0.517,0.4387,-4.3469,-1,NaN
123,0,123,205549,g0001361-144455,10966,10966,158.8,119.1,0.26,36.004,35.379,0.498,0.4002,-14.7487,-1,NaN


In [8]:
print(len(data["6dfgs"]))
data["6dfgs"]

7112


,PGC1_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,Nest_6dfgs
PGC,,,,,
64,72642,36.705,0.551,15324,200033
66,72642,36.379,0.519,14725,200033
114,72642,35.604,0.526,14397,200033
115,115,36.353,0.517,14005,0
123,123,36.004,0.498,10966,205549
...,...,...,...,...,...
4684754,4684754,34.449,0.556,6624,210660
4684803,4684803,35.914,0.512,9189,208156
4684804,4684804,34.731,0.607,9163,208117


In [45]:
catal_dbtable = "ksdssfpdistvpub"
catal = get_catal(catal_dbtable)
catal = catal.rename(columns={"czcmb": "Vcmb"})
data, catalogs = make_data(data, catalogs, catal, "fpsdss", catal_DM="DMc75", catal_eDM="eDM")
catal.head()


scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.

scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/sdss_fp_distances_pub_bar  ../bar_files//.
This catalog has been already imported ...
Use a different key than fpsdss


,Dc75,DMc75,Dc100,DMc100,eDMc,DM100,eDM,logDc,elogDc,skewc,...,r,er,i,ei,s,es,Sn,objid,specid,plate
PGC,,,,,,,,,,,,,,,,,,,,,
20919,204.0,36.548,153.0,35.923,0.458,35.936,0.498,0.040675,0.091698,-0.541032,...,0.55667,0.00434,2.65845,0.00873,2.46123,0.00775,1.0000,1237663547968323716,1951253584033114112,1733
21014,62.6,33.981,46.9,33.356,0.497,33.315,0.508,-0.156026,0.099438,0.076115,...,0.15331,0.00320,2.69511,0.00643,2.30353,0.00511,0.3231,1237663917871988832,2099918005914855936,1865
21072,196.7,36.470,147.6,35.845,0.458,35.858,0.496,0.025511,0.091508,-0.538911,...,0.57642,0.00381,2.63590,0.00767,2.47846,0.00755,1.0000,1237663531326243079,1951295365474969600,1733
21226,208.7,36.597,156.5,35.973,0.488,35.999,0.497,0.034595,0.097532,-0.675872,...,0.47205,0.00345,2.73040,0.00696,2.44156,0.00485,1.0000,1237653588476952852,609190772427745280,541
21473,173.1,36.192,129.8,35.567,0.500,35.584,0.495,-0.023480,0.099913,-0.619854,...,0.70569,0.00368,2.12132,0.00742,2.28253,0.01174,1.0000,1237657595682029975,848939837200295808,754


In [46]:
print(len(data["fpsdss"]))
data["fpsdss"]

34059


,PGC1_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,Nest_fpsdss
PGC,,,,,
20919,20919,36.548,0.498,16184,104087
21014,21014,33.981,0.508,3250,111419
21072,21072,36.470,0.496,14984,104427
21226,21226,36.597,0.497,16290,101391
21473,21493,36.192,0.495,11962,105529
...,...,...,...,...,...
5094035,0,37.500,0.517,29916,0
5094400,52870,36.869,0.499,12259,105205
5094420,3762668,38.803,0.535,22133,0


In [48]:
catal.loc[2180626]

Dc75                    100.9
DMc75                  35.019
Dc100                    75.7
DMc100                 34.395
eDMc                    0.468
DM100                  34.299
eDM                     0.479
logDc                0.027767
elogDc               0.093567
skewc               -0.349573
logD                 0.046916
elogD                0.095898
skew                -0.375417
czh                      7871
eczh                        2
Vcmb                     7909
gczcmb                   7909
Nest                        0
PGC1                        0
IDt17                       0
Ngt17                       1
J2000       J161531.4+412425 
Ra                   243.8807
Dec                    41.407
glon                  65.5758
glat                   46.177
sgl                   71.4499
sgb                   47.1549
gmag                   15.442
egmag                   0.003
rmag                   14.653
ermag                   0.002
rad                     2.732
erad      

In [11]:
df_6dfgs = data["6dfgs"]
df_fpsdss = data["fpsdss"]

In [12]:
catal_dbtable = "kallfp"
df_cf2fp = get_catal(catal_dbtable)

df_cf2fp = df_cf2fp.rename(columns={"1PGC": "PGC1"})


for suffix in ["smc","enr","far"]:
    df_cf2fp = df_cf2fp.rename(columns={"DM"+suffix: "DM_"+suffix})
    df_cf2fp["eDM_"+suffix] = 0.50
    df_cf2fp["DM_"+suffix][df_cf2fp["DM_"+suffix]==0] = np.nan
    df_cf2fp["eDM_"+suffix][df_cf2fp["DM_"+suffix].isna()] = np.nan
    df_cf2fp["PGC1_"+suffix] = df_cf2fp["PGC1"]
    df_cf2fp["Nest_"+suffix] = df_cf2fp["Nest"]
    df_cf2fp["Vcmb_"+suffix] = df_cf2fp["Vcmb"]


del df_cf2fp['PGC1']
del df_cf2fp['Nest']
del df_cf2fp['Vcmb']



scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.

scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/3fp_bar  ../bar_files//.


<ipython-input-12-d73d0456d218>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf2fp["DM_"+suffix][df_cf2fp["DM_"+suffix]==0] = np.nan
<ipython-input-12-d73d0456d218>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cf2fp["eDM_"+suffix][df_cf2fp["DM_"+suffix].isna()] = np.nan


In [13]:
len(df_6dfgs)

7112

In [14]:
how = 'outer'

df = df_6dfgs.join(df_fpsdss, lsuffix='_6dfgs', rsuffix='_fpsdss', how=how)

df = df.join(df_cf2fp, lsuffix='_fp', rsuffix='_cf2fp', how=how)

# df1 =  df[['PGC1_far','eDM_far']]
# df1g = df1.groupby('PGC1_far').count().rename(columns={"eDM_far":"N_far"})
# df1 = df1.reset_index().set_index('PGC1_far').join(df1g, lsuffix='_l', rsuffix='_r', how='left').reset_index().set_index('PGC')
# df1['eDM_far'] = df1['eDM_far']/np.sqrt(df1["N_far"])

# df['eDM_far'] = df1['eDM_far']
# df["N_far"] = df1["N_far"]


for col in df.columns:
    if col.split('_')[0]=='PGC1' and col!='PGC1_fpsdss':
        df['PGC1_fpsdss'] = df['PGC1_fpsdss'].fillna(df[col])
        
for col in df.columns:
    if col.split('_')[0]=='Nest' and col!='Nest_fpsdss':
        df['Nest_fpsdss'] = df['Nest_fpsdss'].fillna(df[col])



catalogs = ["fpsdss", "6dfgs", "smc", "enr", "far"]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat, 'Nest_'+cat, "DM_"+cat, "eDM_"+cat, "Vcmb_"+cat]

df = df[ss]

ss = []
for cat in catalogs:
    ss += ['PGC1_'+cat]   
df['pgc1'] = df[ss].median(axis=1)
df = df[~df['pgc1'].isna()]


print(len(df))

df.head()

42254


,PGC1_fpsdss,Nest_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,PGC1_6dfgs,Nest_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,...,Nest_enr,DM_enr,eDM_enr,Vcmb_enr,PGC1_far,Nest_far,DM_far,eDM_far,Vcmb_far,pgc1
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,NaN,NaN,NaN,72642.0,200033.0,36.705,0.551,15324.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72642.0
66,72642.0,200033.0,NaN,NaN,NaN,72642.0,200033.0,36.379,0.519,14725.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72642.0
114,72642.0,200033.0,NaN,NaN,NaN,72642.0,200033.0,35.604,0.526,14397.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72642.0
115,115.0,0.0,NaN,NaN,NaN,115.0,0.0,36.353,0.517,14005.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.0
123,123.0,205549.0,NaN,NaN,NaN,123.0,205549.0,36.004,0.498,10966.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0


In [15]:
for cat in catalogs:
    df['Vmod_'+cat] = Vcmb2Vmod(df['Vcmb_'+cat])
    logD = (df['DM_'+cat]-25)/5.
    df['logH_'+cat] = np.log10(df['Vmod_'+cat]) - logD 
    
    print(cat, 10**df['logH_'+cat][df['Vmod_'+cat]>4000].median())

fpsdss 73.80770083447044
6dfgs 73.29244972001047
smc 76.73704799488011
enr 76.22256166035713
far 80.36152475092275


In [16]:
sigmaClips = {}

for i in range(1):
    for cat in catalogs:

        logH = df['logH_'+cat]
        med = logH[df['Vmod_'+cat]>4000].median()
        stdev = logH[df['Vmod_'+cat]>4000].std()
        
#         print cat, 10**med
        
        if not cat in sigmaClips:
                sigmaClips[cat] = []
        sigmaClips[cat] += list(df[(logH > med+3.5*stdev) | (logH < med-3.5*stdev)].index.values)
        df['DM_'+cat][(logH > med+3.5*stdev) | (logH < med-3.5*stdev)] = np.nan
        df['eDM_'+cat][(logH > med+3.5*stdev) | (logH < med-3.5*stdev)] = np.nan
        df['logH_'+cat][(logH > med+3.5*stdev) | (logH < med-3.5*stdev)] = np.nan


In [17]:
myDict = {}
myDict['fpsdss'] = 0

for cat in catalogs[1:]:

        delta = df['DM_fpsdss'] - df['DM_'+cat]

        stdev = delta.std()
        med   = delta.median()
        myDict[cat] = med


In [18]:
for cat in sigmaClips:
    print(cat, len(sigmaClips[cat]))

myDict

fpsdss 14
6dfgs 14
smc 1
enr 2
far 0


{'fpsdss': 0,
 '6dfgs': -0.15200000000000102,
 'smc': -0.06899999999999551,
 'enr': -0.16100000000000136,
 'far': 0.02499999999999858}

In [19]:
ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
print(len(df["PGC1_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
    df["PGC1_fpsdss"][ind] = df["PGC1_"+cat][ind]
    

ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
print(len(df["PGC1_fpsdss"][ind]))

ind = (df["PGC1_fpsdss"].isna()) | (df["PGC1_fpsdss"]==0)
df["PGC1_fpsdss"][ind] = np.nan

8934
8928


In [20]:
ind = (df["Nest_fpsdss"].isna()) | (df["Nest_fpsdss"]==0)
print(len(df["Nest_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["Nest_fpsdss"].isna()) | (df["Nest_fpsdss"]==0)
    df["Nest_fpsdss"][ind] = df["Nest_"+cat][ind]
    
ind = (df["Nest_fpsdss"].isna()) | (df["Nest_fpsdss"]==0)
df["Nest_fpsdss"][ind] = np.nan

27544


In [21]:
ind = (df["Vmod_fpsdss"].isna()) | (df["Vmod_fpsdss"]==0)
print(len(df["Vmod_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["Vmod_fpsdss"].isna()) | (df["Vmod_fpsdss"]==0)
    df["Vmod_fpsdss"][ind] = df["Vmod_"+cat][ind]
    
ind = (df["Vmod_fpsdss"].isna()) | (df["Vmod_fpsdss"]==0)
df["Vmod_fpsdss"][ind] = np.nan

8195


In [22]:
ind = (df["Vcmb_fpsdss"].isna()) | (df["Vcmb_fpsdss"]==0)
print(len(df["Vcmb_fpsdss"][ind]))

for cat in catalogs:
    ind = (df["Vcmb_fpsdss"].isna()) | (df["Vcmb_fpsdss"]==0)
    df["Vcmb_fpsdss"][ind] = df["Vcmb_"+cat][ind]
    
ind = (df["Vcmb_fpsdss"].isna()) | (df["Vcmb_fpsdss"]==0)
df["Vcmb_fpsdss"][ind] = np.nan

8195


In [23]:
df.head()

,PGC1_fpsdss,Nest_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,PGC1_6dfgs,Nest_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,...,Vmod_fpsdss,logH_fpsdss,Vmod_6dfgs,logH_6dfgs,Vmod_smc,logH_smc,Vmod_enr,logH_enr,Vmod_far,logH_far
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,NaN,NaN,15324.0,72642.0,200033.0,36.705,0.551,15324.0,...,15938.430745,NaN,15938.430745,1.861446,NaN,NaN,NaN,NaN,NaN,NaN
66,72642.0,200033.0,NaN,NaN,14725.0,72642.0,200033.0,36.379,0.519,14725.0,...,15292.703769,NaN,15292.703769,1.908684,NaN,NaN,NaN,NaN,NaN,NaN
114,72642.0,200033.0,NaN,NaN,14397.0,72642.0,200033.0,35.604,0.526,14397.0,...,14939.887452,NaN,14939.887452,2.053547,NaN,NaN,NaN,NaN,NaN,NaN
115,115.0,NaN,NaN,NaN,14005.0,115.0,0.0,36.353,0.517,14005.0,...,14518.945116,NaN,14518.945116,1.891335,NaN,NaN,NaN,NaN,NaN,NaN
123,123.0,205549.0,NaN,NaN,10966.0,123.0,205549.0,36.004,0.498,10966.0,...,11282.137786,NaN,11282.137786,1.851591,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
myDict = {}
myDict['fpsdss'] = 0

for cat in catalogs[1:]:

        delta = df['DM_fpsdss'] - df['DM_'+cat]

        stdev = delta.std()
        med   = delta.median()
        myDict[cat] = med


In [25]:
for cat in sigmaClips:
    print(cat, len(sigmaClips[cat]))

myDict

fpsdss 14
6dfgs 14
smc 1
enr 2
far 0


{'fpsdss': 0,
 '6dfgs': -0.15200000000000102,
 'smc': -0.06899999999999551,
 'enr': -0.16100000000000136,
 'far': 0.02499999999999858}

In [26]:
cat = catalogs[0]
N = len(sigmaClips[cat])
x = [1 for i in range(N)]
Dreject = pd.DataFrame(list(zip(sigmaClips[cat], x)), columns=['PGC',cat])
Dreject = Dreject.set_index('PGC')

for cat in catalogs[1:]:
    x = [1 for i in range(N)]
    Dj = pd.DataFrame(list(zip(sigmaClips[cat], x)), columns=['PGC',cat])
    Dj = Dj.set_index('PGC')
    Dreject = Dreject.join(Dj, how='outer')

Dreject = Dreject.fillna(0).astype('int')
Dreject = Dreject.reset_index()
Dreject.head(20)

import pandas as pd
from tabulate import tabulate

content = tabulate(Dreject.values.tolist(), list(Dreject.columns), tablefmt="pipe")
open('3SigmaClips_FP.csv', "w").write(content)

1880

In [27]:
sig3 = pd.read_csv('3SigmaClips_FP.csv', delimiter='|')
sig3 = sig3.rename(columns=lambda x: x.strip())
new_col = {}
for cat in catalogs:
    new_col[cat] = cat+'_flag'

sig3 = sig3.drop(index=0).set_index('PGC')[catalogs]
sig3 = sig3.rename(columns=new_col)
sig3 = sig3.astype('int32')
sig3.index = sig3.index.astype('int32')

sig3.head(10)

,fpsdss_flag,6dfgs_flag,smc_flag,enr_flag,far_flag
PGC,,,,,
15472,0,1,0,0,0
31765,0,0,0,1,0
40653,0,0,0,1,0
43506,0,1,0,0,0
49131,0,0,1,0,0
56712,1,0,0,0,0
66351,0,1,0,0,0
84493,1,0,0,0,0
160575,0,1,0,0,0


In [28]:
len(sig3[sig3["fpsdss_flag"]==1])

14

In [29]:
df.index = df.index.astype('int32')
df = df.join(sig3, lsuffix='_l', rsuffix='_r', how="outer")

for cat in catalogs:
    ind = df[cat+"_flag"].isna()
    df[cat+"_flag"][ind] = 0

df.head()

,PGC1_fpsdss,Nest_fpsdss,DM_fpsdss,eDM_fpsdss,Vcmb_fpsdss,PGC1_6dfgs,Nest_6dfgs,DM_6dfgs,eDM_6dfgs,Vcmb_6dfgs,...,logH_smc,Vmod_enr,logH_enr,Vmod_far,logH_far,fpsdss_flag,6dfgs_flag,smc_flag,enr_flag,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,NaN,NaN,15324.0,72642.0,200033.0,36.705,0.551,15324.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
66,72642.0,200033.0,NaN,NaN,14725.0,72642.0,200033.0,36.379,0.519,14725.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
114,72642.0,200033.0,NaN,NaN,14397.0,72642.0,200033.0,35.604,0.526,14397.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
115,115.0,NaN,NaN,NaN,14005.0,115.0,0.0,36.353,0.517,14005.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
123,123.0,205549.0,NaN,NaN,10966.0,123.0,205549.0,36.004,0.498,10966.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [30]:
myDict = {}
myDict["fpsdss"] = 0
myDict["6dfgs"] = -0.083
myDict["smc"] = 0.038
myDict["enr"] = 0.006
myDict["far"] = 0.161


for cat in catalogs:
    df['DM_'+cat] += myDict[cat]
    df['w_'+cat] = (1.-df[cat+'_flag'])/df['eDM_'+cat]**2
    df['w_'+cat] = df['w_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['w_'+cat] = df.apply(lambda x: x['w_'+cat] if x[cat+'_flag']==0 else np.nan, axis=1)
    df['e_'+cat] = (1.-df[cat+'_flag'])*df['eDM_'+cat]
    df['e_'+cat] = df['e_'+cat].apply(lambda x: x if x!=0 else np.nan)
    df['e_'+cat] = df.apply(lambda x: x['e_'+cat] if x[cat+'_flag']==0 else np.nan, axis=1)
    df['xw_'+cat] = (df['DM_'+cat])*df['w_'+cat]
    df['xw_'+cat] = df.apply(lambda x: x['xw_'+cat] if x[cat+'_flag']==0 else np.nan, axis=1)

sw = ['w_'+cat for cat in catalogs]
ee = ['e_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)

df["eDM_av"] = df[ee].min(axis=1) # np.sqrt(Err2)
df["DM_av"] = df[sx].sum(axis=1)*Err2   

In [31]:
df['av_flag'] = 0*df['DM_av']
df['av_flag'][df['DM_av'].isna()] = 1

In [32]:
for cat in catalogs:
    df['w_'+cat] = 1./df['eDM_'+cat]**2
    df['xw_'+cat] = df['DM_'+cat]*df['w_'+cat]
    df['e_'+cat] = 1.*df['eDM_'+cat]

sw = ['w_'+cat for cat in catalogs]
ee = ['e_'+cat for cat in catalogs]
sx = ['xw_'+cat for cat in catalogs]

Err2 = 1./df[sw].sum(axis=1)
df["DM_av_"] = df[sx].sum(axis=1)*Err2
df["eDM_av_"] = df[ee].min(axis=1)  #  np.sqrt(Err2)

ind = df["av_flag"] == 1
df["eDM_av"][ind] = df["eDM_av_"][ind]
df["DM_av"][ind] = df["DM_av_"][ind]

In [33]:
df[ee+["eDM_av"]].head()

,e_fpsdss,e_6dfgs,e_smc,e_enr,e_far,eDM_av
PGC,,,,,,
64,NaN,0.551,NaN,NaN,NaN,0.551
66,NaN,0.519,NaN,NaN,NaN,0.519
114,NaN,0.526,NaN,NaN,NaN,0.526
115,NaN,0.517,NaN,NaN,NaN,0.517
123,NaN,0.498,NaN,NaN,NaN,0.498


In [34]:
ss = ['PGC1_fpsdss', 'Nest_fpsdss', 'Vcmb_fpsdss', 'Vmod_fpsdss', "DM_av", "eDM_av", "av_flag"]
for cat in catalogs:
    ss += ["DM_"+cat, "eDM_"+cat, cat+'_flag']

df = df.fillna(0)

df = df[ss]


df = df.rename(columns={'PGC1_fpsdss': 'PGC1',
                        'Nest_fpsdss': 'Nest',
                        'Vcmb_fpsdss': 'Vcmb',
                        'Vmod_fpsdss': 'Vmod'
                       })

df.head()

,PGC1,Nest,Vcmb,Vmod,DM_av,eDM_av,av_flag,DM_fpsdss,eDM_fpsdss,fpsdss_flag,...,6dfgs_flag,DM_smc,eDM_smc,smc_flag,DM_enr,eDM_enr,enr_flag,DM_far,eDM_far,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,15324.0,15938.430745,36.622,0.551,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,72642.0,200033.0,14725.0,15292.703769,36.296,0.519,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,72642.0,200033.0,14397.0,14939.887452,35.521,0.526,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,115.0,0.0,14005.0,14518.945116,36.270,0.517,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,123.0,205549.0,10966.0,11282.137786,35.921,0.498,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
np.max(df.DM_fpsdss.values)

39.657

In [36]:
for cat in catalogs:
    edm = 'eDM_'+cat
    dm = 'DM_'+cat
    flag = cat+'_flag'
      
    print(dm, edm, flag)
    
    df[edm] = df.apply(lambda x: x[edm] if x[dm]!=0 else np.nan, axis=1)
    df[flag] = df.apply(lambda x: x[flag] if x[dm]!=0 else np.nan, axis=1)
    df[dm] = df.apply(lambda x: x[dm] if x[dm]!=0 else np.nan, axis=1)
    
    
    ### COMMET THIS unless SNIa
    ## don't perform the adjsutments on the individual cataloged distances
#     df['DM_'+cat] -= myDict[cat]

DM_fpsdss eDM_fpsdss fpsdss_flag
DM_6dfgs eDM_6dfgs 6dfgs_flag
DM_smc eDM_smc smc_flag
DM_enr eDM_enr enr_flag
DM_far eDM_far far_flag


In [37]:
df.head(20)

,PGC1,Nest,Vcmb,Vmod,DM_av,eDM_av,av_flag,DM_fpsdss,eDM_fpsdss,fpsdss_flag,...,6dfgs_flag,DM_smc,eDM_smc,smc_flag,DM_enr,eDM_enr,enr_flag,DM_far,eDM_far,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,72642.0,200033.0,15324.0,15938.430745,36.622,0.551,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,72642.0,200033.0,14725.0,15292.703769,36.296,0.519,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,72642.0,200033.0,14397.0,14939.887452,35.521,0.526,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,115.0,0.0,14005.0,14518.945116,36.270,0.517,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,123.0,205549.0,10966.0,11282.137786,35.921,0.498,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,75.0,200697.0,11615.0,11969.416136,35.865,0.507,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,72642.0,200033.0,14365.0,14905.495569,36.611,0.529,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,382.0,200108.0,10147.0,10417.919123,35.970,0.478,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,262.0,208331.0,8795.0,8998.830832,35.473,0.555,0.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
catal_dbtable = "kcf4fps"
df_EDD = get_catal(catal_dbtable)

df_EDD.head()


scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.

scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/FP_CF4_components_bar  ../bar_files//.


,T17,PGC1,Nest,Vcmb,Vmod,DM_zp,DM_av,eDM,av_flag,DM_fpsdss,...,6dfgs_flag,DM_smac,eDM_smac,smac_flag,DM_enear,eDM_enear,enear_flag,DM_efar,eDM_efar,efar_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,0,72642,200033,15324,15938,36.606,36.63,0.63,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,0,72642,200033,14725,15292,36.276,36.30,0.59,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,0,72642,200033,14397,14939,35.496,35.52,0.59,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,0,115,0,14005,14518,36.246,36.27,0.59,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,0,123,205549,10966,11282,35.886,35.91,0.56,0,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
print(len(df_fpsdss))

# his updated in the All CF4 FP Samples file in EDD

df = df_EDD[["T17"]].join(df, how="inner")

print(len(df_fpsdss))

df.head()

34059
34059


,T17,PGC1,Nest,Vcmb,Vmod,DM_av,eDM_av,av_flag,DM_fpsdss,eDM_fpsdss,...,6dfgs_flag,DM_smc,eDM_smc,smc_flag,DM_enr,eDM_enr,enr_flag,DM_far,eDM_far,far_flag
PGC,,,,,,,,,,,,,,,,,,,,,
64,0,72642.0,200033.0,15324.0,15938.430745,36.622,0.551,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,0,72642.0,200033.0,14725.0,15292.703769,36.296,0.519,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,0,72642.0,200033.0,14397.0,14939.887452,35.521,0.526,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,0,115.0,0.0,14005.0,14518.945116,36.270,0.517,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,0,123.0,205549.0,10966.0,11282.137786,35.921,0.498,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.to_csv("FP_CF4_components_bar.csv", sep='|')

In [41]:
table   = np.genfromtxt('FP_CF4_components_bar.csv' , delimiter='|', 
                        filling_values=-10000, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["PGC", "PGC1", "Nest", "T17"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]+["av_flag","fpsdss_flag","6dfgs_flag","smc_flag","sfi_flag","enr_flag","far_flag"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.2f'))

## to be used on EDD
myTable.write('FP_CF4_components_bar_221027', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)

In [42]:
len(df)

42221